# Cellcycle

In [16]:
library_load <- suppressMessages(
    
    suppressWarnings(
    
        list(

            # Seurat 
            library(Seurat), 
            
            # Module score 
            library(UCell), 

            # SingleR
            library(SingleR), 
            library(SingleCellExperiment), 
            
            # Reference
            library(celldex), 

            # Data 
            library(tidyverse), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [17]:
# Configure reticulate 
use_condaenv(condaenv='p.3.9.19-FD20200109SPLENO', conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

python:         /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/bin/python
libpython:      /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/lib/libpython3.9.so
pythonhome:     /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO:/nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO
version:        3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
numpy:          /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/lib/python3.9/site-packages/numpy
numpy_version:  1.26.4

NOTE: Python version was forced by use_python() function

In [18]:
options(warn=-1)

In [19]:
random_seed <- 42
set.seed(random_seed)

In [20]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [21]:
# Source files
source("plotting_global.R")

# Parameter settings

In [22]:
# Files 
raw_rds_file <- "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/raw.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import Seurat object

In [23]:
so <- readRDS(raw_rds_file)

In [24]:
so <- subset(so, subset=nCount_RNA>=1000)
so <- subset(so, subset=pMt_RNA<=15)

In [25]:
cnt <- GetAssayData(so, assay="RNA", slot="counts")
cnt <- cnt[rowSums(cnt>=1)>=3, ]
so <- CreateSeuratObject(counts=cnt, meta.data=so@meta.data, project="FD20200109SPLENO")

In [26]:
so <- NormalizeData(so, normalization.method="LogNormalize", verbose=FALSE)

# Human to mouse gene symbol convert

In [27]:
# Seurat cell cycle genes 
cc_genes_s <- cc.genes.updated.2019$s.genes
cc_genes_g2m <- cc.genes.updated.2019$g2m.genes

# Get mouse orthologs from human gene simbols
httr::set_config(httr::config(ssl_verifypeer=FALSE))

hgnc_mart <- biomaRt::useMart("ensembl", dataset="hsapiens_gene_ensembl", host="https://dec2021.archive.ensembl.org/")
mm_mart <- biomaRt::useMart("ensembl", dataset="mmusculus_gene_ensembl", host="https://dec2021.archive.ensembl.org/")

# Seurat gene set based on Tirosh et al, 2015

In [28]:
# Conver genes
cc_genes_s <- biomaRt::getLDS(attributes=c("hgnc_symbol"), filters="hgnc_symbol", values=cc_genes_s, mart=hgnc_mart, attributesL=c("mgi_symbol"), martL=mm_mart, uniqueRows=TRUE)[, 2]
cc_genes_g2m <- biomaRt::getLDS(attributes=c("hgnc_symbol"), filters="hgnc_symbol", values=cc_genes_g2m, mart=hgnc_mart, attributesL=c("mgi_symbol"), martL=mm_mart, uniqueRows=TRUE)[, 2]

# Compute Module score with UCell 

In [29]:
# Cell cycle scoring anc class
so <- AddModuleScore_UCell(so, features=list(msS_RNA=cc_genes_s, msG2M_RNA=cc_genes_g2m), assay="RNA", slot="counts", name="")

# MinMax scale per sample

In [ ]:
minmax <- function(x) {
    
    x <- (x-min(x))/(max(x)-min(x))
    
    return(x)

}

In [ ]:
# Minmax scale cellcycle phase scores per sample 
so$msS_scale_RNA <- dplyr::group_by(so@meta.data, sample_name) %>% dplyr::mutate(msS_scale_RNA=minmax(msS_RNA)) %>% dplyr::pull(msS_scale_RNA)
so$msG2M_scale_RNA <- dplyr::group_by(so@meta.data, sample_name) %>% dplyr::mutate(msG2M_scale_RNA=minmax(msG2M_RNA)) %>% dplyr::pull(msG2M_scale_RNA)

# Set cell cycle class

In [ ]:
so$msCC_class_RNA <- ifelse(so$msS_scale_RNA >= 0.25 & so$msS_scale_RNA > so$msG2M_scale_RNA, "S", ifelse(so$msG2M_scale_RNA >= 0.25 & so$msG2M_scale_RNA > so$msS_scale_RNA, "G2M", "G1"))
so$msCC_class_RNA <- factor(so$msCC_class_RNA, levels = c("S", "G2M", "G1"))

In [ ]:
options(repr.plot.width=15, repr.plot.height=7.5)

ggplot(so@meta.data, aes(x=msG2M_scale_RNA, y=msS_scale_RNA, color=msCC_class_RNA)) + geom_point() + facet_wrap(~sample_name, ncol=4) + theme(aspect.ratio=1) + scale_color_manual(values=color$msCC_class_RNA)

# Cellcycle difference 

In [ ]:
so$msCC_scale_RNA <- so$msS_scale_RNA - so$msG2M_scale_RNA

In [ ]:
options(repr.plot.width=15, repr.plot.height=5)

ggplot(so@meta.data, aes(x=sample_name, y=msCC_scale_RNA, color=msCC_class_RNA)) + geom_jitter() + scale_color_manual(values=color$msCC_class_RNA) + theme(axis.text.x = element_text(angle=90, vjust=0.5, hjust=1))

# Save results 

In [ ]:
write.csv(so@meta.data[, c("msS_RNA", "msG2M_RNA", "msS_scale_RNA", "msG2M_scale_RNA", "msCC_scale_RNA")], "result/cellcycle/cellcycle.csv", row.names=TRUE)

# Session info

In [ ]:
sessionInfo()